In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import opensmile
import librosa

In [2]:
path=Path(os.getcwd()).parent
DIR_PATH=str(path) + '/'

from utils import AUD_FILE_PATH
audio_files=AUD_FILE_PATH.glob('*.mp3')
audio_file_names=[i.stem for i in audio_files] 
audio_file_names.sort()

In [3]:
len(audio_file_names)

206

In [8]:
dur = []
for i,audio_file in enumerate(audio_file_names):
    splitted_name = audio_file.split('-')  #['headset', 'audio', 'N249', 'planning', '1', 'child', '1', 'processed']

    x, sr = librosa.load(Path.joinpath(AUD_FILE_PATH, audio_file + '.mp3'))
    
    dur.append( x.shape[0]/sr)
 
print(np.average(dur), np.std(dur))

135.25934922836447 91.92576267449647


In [ ]:
from .utils import PROCESSED_AUD_FILE_PATH, FEATURE_SETS

summary_stat = None 
custom_window_length_factor = None
feature_set_name = 'emob'
# --------------------------- Reading the processed audio file names from disk ------------------------ #    
audio_files = PROCESSED_AUD_FILE_PATH.glob("*.mp3")
audio_file_names = [i.stem for i in audio_files]
audio_file_names.sort()

child_link_csv = pd.read_csv('/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/headset-audio/vidfile_link.csv')


if summary_stat is None:
    
    if custom_window_length_factor is None:
        extracted_data_savepath = DIR_PATH / f"extracted-data/raw-extract/{feature_set_name}-extract.csv"
    else:
        extracted_data_savepath = DIR_PATH / f"extracted-data/raw-extract/{feature_set_name}-cwlf({custom_window_length_factor})-extract.csv"
    
    agg_aud_df = pd.DataFrame()
    if extracted_data_savepath.is_file():
        print('Data already exists in disk, reading..\n')
        agg_aud_df = pd.read_csv(extracted_data_savepath, usecols=features+['pair_id','round','child_id','condition'])    
        return agg_aud_df
    
    else:
        all_features = FEATURE_SETS[feature_set_name]['features'](extractor)
        print('Data does not exist in disk, reading audio files and extracting to a csv..\n')
        for audio_file in audio_file_names:
            # ['headset', 'audio', 'N249', 'planning', '1', 'child', '1', 'processed']
            splitted_name = audio_file.split("-")
            pair_id = splitted_name[2]
            round = int(splitted_name[4])
            child = int(splitted_name[6])
            child_id = child_link_csv['child_id'].loc[ (child_link_csv['pair_id']==pair_id) & (child_link_csv['round']==round) & (child_link_csv['child']==child) ].reset_index(drop=True)[0] #get child_id from 'vidfile_link.csv'

            x, sr = librosa.load(
                Path.joinpath(PROCESSED_AUD_FILE_PATH, audio_file + ".mp3")
            )

            if custom_window_length_factor == None:
                aud_df = pd.DataFrame(extractor.process_signal(x, sr))
            else:
                aud_df = pd.DataFrame(extractor.process_signal(x, sr * custom_window_length_factor))

            feature_outputs = aud_df[(col for col in all_features)].astype(float).reset_index(drop=True)
            feature_outputs['pair_id']= pair_id
            feature_outputs['round']= round
            feature_outputs['child_id']= child_id
            feature_outputs["condition"] = "positive" if (pair_id[0] == "P") else "negative"

            agg_aud_df = pd.concat([agg_aud_df, feature_outputs]).reset_index(drop=True)





def extract_raw_data(extractor, feature_set_name, features, summary_stat = None, custom_window_length_factor = None, timestamp = False):

# --------------------------- Reading the processed audio file names from disk ------------------------ #    
    audio_files = PROCESSED_AUD_FILE_PATH.glob("*.mp3")
    audio_file_names = [i.stem for i in audio_files]
    audio_file_names.sort()
    
    child_link_csv = pd.read_csv('/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/headset-audio/vidfile_link.csv')


    if summary_stat is None:
        
        if custom_window_length_factor is None:
            extracted_data_savepath = DIR_PATH / f"extracted-data/raw-extract/{feature_set_name}-extract.csv"
        else:
            extracted_data_savepath = DIR_PATH / f"extracted-data/raw-extract/{feature_set_name}-cwlf({custom_window_length_factor})-extract.csv"
        
        agg_aud_df = pd.DataFrame()
        if extracted_data_savepath.is_file():
            print('Data already exists in disk, reading..\n')
            agg_aud_df = pd.read_csv(extracted_data_savepath, usecols=features+['pair_id','round','child_id','condition'])    
            return agg_aud_df
        
        else:
            all_features = FEATURE_SETS[feature_set_name]['features'](extractor)
            print('Data does not exist in disk, reading audio files and extracting to a csv..\n')
            for audio_file in audio_file_names:
                # ['headset', 'audio', 'N249', 'planning', '1', 'child', '1', 'processed']
                splitted_name = audio_file.split("-")
                pair_id = splitted_name[2]
                round = int(splitted_name[4])
                child = int(splitted_name[6])
                child_id = child_link_csv['child_id'].loc[ (child_link_csv['pair_id']==pair_id) & (child_link_csv['round']==round) & (child_link_csv['child']==child) ].reset_index(drop=True)[0] #get child_id from 'vidfile_link.csv'

                x, sr = librosa.load(
                    Path.joinpath(PROCESSED_AUD_FILE_PATH, audio_file + ".mp3")
                )

                if custom_window_length_factor == None:
                    aud_df = pd.DataFrame(extractor.process_signal(x, sr))
                else:
                    aud_df = pd.DataFrame(extractor.process_signal(x, sr * custom_window_length_factor))

                feature_outputs = aud_df[(col for col in all_features)].astype(float).reset_index(drop=True)
                feature_outputs['pair_id']= pair_id
                feature_outputs['round']= round
                feature_outputs['child_id']= child_id
                feature_outputs["condition"] = "positive" if (pair_id[0] == "P") else "negative"

                agg_aud_df = pd.concat([agg_aud_df, feature_outputs]).reset_index(drop=True)

            agg_aud_df.to_csv(extracted_data_savepath, index=False)
            
            print(f'Extracted feature csv {extracted_data_savepath} saved successfully!\n')

            return agg_aud_df
    else:
        if not os.path.exists(DIR_PATH / f"extracted-data/summary-statistics-{summary_stat}"):
            os.makedirs(DIR_PATH / f"extracted-data/summary-statistics-{summary_stat}")
        
        if custom_window_length_factor is None:
            extracted_data_savepath = DIR_PATH / f"extracted-data/summary-statistics-{summary_stat}/{feature_set_name}-extract.csv"
        else:
            extracted_data_savepath = DIR_PATH / f"extracted-data/summary-statistics-{summary_stat}/{feature_set_name}-cwlf({custom_window_length_factor})-extract.csv"
        
        

        if extracted_data_savepath.is_file():
            print('Data already exists in disk, reading..\n')
            agg_aud_df = pd.read_csv(extracted_data_savepath)    
            return agg_aud_df
        
        else:
            print('Data does not exist in disk, reading audio files and extracting to a csv..\n')
            all_features = FEATURE_SETS[feature_set_name]['features'](extractor)
            print(all_features)
            agg_aud_df = pd.DataFrame(columns=['pair_id','round', 'child_id', 'condition'] + all_features)

            for audio_file in audio_file_names:
                # ['headset', 'audio', 'N249', 'planning', '1', 'child', '1', 'processed']
                splitted_name = audio_file.split("-")
                pair_id = splitted_name[2]
                round = int(splitted_name[4])
                child = int(splitted_name[6])
                child_id = child_link_csv['child_id'].loc[ (child_link_csv['pair_id']==pair_id) & (child_link_csv['round']==round) & (child_link_csv['child']==child) ].reset_index(drop=True)[0] #get child_id from 'vidfile_link.csv'
                condition = "positive" if (pair_id[0] == "P") else "negative"

                x, sr = librosa.load(
                    Path.joinpath(PROCESSED_AUD_FILE_PATH, audio_file + ".mp3")
                )
                
                if custom_window_length_factor == None:
                    aud_df = pd.DataFrame(extractor.process_signal(x, sr))
                else:
                    aud_df = pd.DataFrame(extractor.process_signal(x, sr * custom_window_length_factor))

                agg_aud_df.loc[len(agg_aud_df)] = [splitted_name[2], splitted_name[4], child_id, condition ] + aud_df[ all_features ].describe().loc[summary_stat].to_list()   #.mean().to_list()


            agg_aud_df.to_csv(extracted_data_savepath, index=False)
            
            print(f'Extracted feature csv {extracted_data_savepath} saved successfully!\n')

            return agg_aud_df    

In [3]:
import joblib

In [3]:
extractor = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    multiprocessing=True,
    logfile='log.txt'
)

extractor2 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    multiprocessing=True,
    logfile='log.txt'
)

extractor3 = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
    multiprocessing=True,
    logfile='log.txt'
)

In [ ]:
config_file = DIR_PATH + 'binary-classification-audio/opensmile-config/emo_IS09.conf'
ex = extractor = opensmile.Smile(
    feature_set= config_file,
    multiprocessing=True,
    logfile='log1.txt'
)

In [5]:
extractor2.feature_names

['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3amplitudeLogRelF0_sma3nz']

In [6]:
extractor.feature_names

['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'spectralFlux_sma3',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2bandwidth_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3bandwidth_sma3nz',
 'F3amplitudeLogRelF0_sma3nz']

In [8]:
extractor3.feature_names

['pcm_intensity_sma',
 'pcm_loudness_sma',
 'mfcc_sma[1]',
 'mfcc_sma[2]',
 'mfcc_sma[3]',
 'mfcc_sma[4]',
 'mfcc_sma[5]',
 'mfcc_sma[6]',
 'mfcc_sma[7]',
 'mfcc_sma[8]',
 'mfcc_sma[9]',
 'mfcc_sma[10]',
 'mfcc_sma[11]',
 'mfcc_sma[12]',
 'lspFreq_sma[0]',
 'lspFreq_sma[1]',
 'lspFreq_sma[2]',
 'lspFreq_sma[3]',
 'lspFreq_sma[4]',
 'lspFreq_sma[5]',
 'lspFreq_sma[6]',
 'lspFreq_sma[7]',
 'pcm_zcr_sma',
 'voiceProb_sma',
 'F0_sma',
 'F0env_sma']

In [149]:
feature_set = extractor.feature_names[-13:]
feature_set[-13:], len(feature_set[-13:]) 

(['shimmerLocaldB_sma3nz',
  'HNRdBACF_sma3nz',
  'logRelF0-H1-H2_sma3nz',
  'logRelF0-H1-A3_sma3nz',
  'F1frequency_sma3nz',
  'F1bandwidth_sma3nz',
  'F1amplitudeLogRelF0_sma3nz',
  'F2frequency_sma3nz',
  'F2bandwidth_sma3nz',
  'F2amplitudeLogRelF0_sma3nz',
  'F3frequency_sma3nz',
  'F3bandwidth_sma3nz',
  'F3amplitudeLogRelF0_sma3nz'],
 13)

In [153]:
extractor.feature_names

['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'spectralFlux_sma3',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2bandwidth_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3bandwidth_sma3nz',
 'F3amplitudeLogRelF0_sma3nz']

In [151]:
extractor2.feature_names

['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3amplitudeLogRelF0_sma3nz']

In [144]:
#set.intersection(set(extractor.feature_names), set(extractor2.feature_names)) #-  
#set.union(set(extractor.feature_names), set(extractor2.feature_names))
set(extractor.feature_names) - set.intersection(set(extractor.feature_names), set(extractor2.feature_names))

{'mfcc1_sma3', 'mfcc2_sma3', 'mfcc3_sma3', 'mfcc4_sma3', 'spectralFlux_sma3'}

In [130]:
FEATURE_SETS = {
    "GeM_5": {
        "extractor": opensmile.Smile(
            feature_set=opensmile.FeatureSet.GeMAPSv01b,
            feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
            multiprocessing=True,
        ),
        "features": lambda x: x.feature_names[:5],
    },
    "GeM_13": {
        "extractor": opensmile.Smile(
            feature_set=opensmile.FeatureSet.GeMAPSv01b,
            feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
            multiprocessing=True,
        ),
        "features": lambda x: x.feature_names[-13:],
    },
    "eGe_10": {
        "extractor": opensmile.Smile(
            feature_set=opensmile.FeatureSet.eGeMAPSv02,
            feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
            multiprocessing=True,
        ),
        "features": lambda x: x.feature_names[:10],
    },
    "eGe_13": {
        "extractor": opensmile.Smile(
            feature_set=opensmile.FeatureSet.eGeMAPSv02,
            feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
            multiprocessing=True,
        ),
        "features": lambda x: x.feature_names[-13:],
    },
}

In [147]:
for key, item in FEATURE_SETS.items():
    print(key)
    for i in FEATURE_SETS[key]['features'] (FEATURE_SETS[key]['extractor']):
        print(i)
    print(len(FEATURE_SETS[key]['features'] (FEATURE_SETS[key]['extractor'])))
    print('\n')

GeM_5
Loudness_sma3
alphaRatio_sma3
hammarbergIndex_sma3
slope0-500_sma3
slope500-1500_sma3
5


GeM_13
F0semitoneFrom27.5Hz_sma3nz
jitterLocal_sma3nz
shimmerLocaldB_sma3nz
HNRdBACF_sma3nz
logRelF0-H1-H2_sma3nz
logRelF0-H1-A3_sma3nz
F1frequency_sma3nz
F1bandwidth_sma3nz
F1amplitudeLogRelF0_sma3nz
F2frequency_sma3nz
F2amplitudeLogRelF0_sma3nz
F3frequency_sma3nz
F3amplitudeLogRelF0_sma3nz
13


eGe_10
Loudness_sma3
alphaRatio_sma3
hammarbergIndex_sma3
slope0-500_sma3
slope500-1500_sma3
spectralFlux_sma3
mfcc1_sma3
mfcc2_sma3
mfcc3_sma3
mfcc4_sma3
10


eGe_13
shimmerLocaldB_sma3nz
HNRdBACF_sma3nz
logRelF0-H1-H2_sma3nz
logRelF0-H1-A3_sma3nz
F1frequency_sma3nz
F1bandwidth_sma3nz
F1amplitudeLogRelF0_sma3nz
F2frequency_sma3nz
F2bandwidth_sma3nz
F2amplitudeLogRelF0_sma3nz
F3frequency_sma3nz
F3bandwidth_sma3nz
F3amplitudeLogRelF0_sma3nz
13




In [99]:
[FEATURE_SETS['GeM_5']['extractor']]

[{'$opensmile.core.smile.Smile': {'feature_set': 'GeMAPSv01b', 'feature_level': 'LowLevelDescriptors', 'options': {}, 'sampling_rate': None, 'channels': [0], 'mixdown': False, 'resample': False}}]

In [15]:
aud_file = '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_audio/processed-data/headset-audio-N249-planning-1-child-1-processed.mp3' #
x , sr = librosa.load(aud_file, sr=None)

In [16]:
x, sr

(array([ 0.        ,  0.        ,  0.        , ..., -0.01211856,
        -0.01241115, -0.01164646], dtype=float32),
 22050)

In [17]:
df = pd.DataFrame(extractor.process_signal(x,sr*4))

In [18]:
df['timestamp'] = [x*0.04 + 3.48 for x in range(0,int(df.shape[0]))]

In [19]:
df.tail(10)

,,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz,timestamp
start,end,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:41.210000,0 days 00:00:41.230000,0.091836,17.999617,6.211651,0.089621,0.035366,0.030107,-11.477993,-27.246155,4.712319,9.042962,...,997.883667,1430.647949,0.000000,1839.599609,1315.593384,56.847443,2776.061279,1448.654297,56.725624,168.32
0 days 00:00:41.220000,0 days 00:00:41.240000,0.521534,33.741863,2.104679,0.059391,0.056354,0.479247,-25.965319,-45.110672,8.105739,14.229150,...,1145.669312,1146.700317,55.104275,1989.535156,1231.934448,59.918575,2945.615967,1186.636841,55.377914,168.36
0 days 00:00:41.230000,0 days 00:00:41.250000,0.975796,42.620728,-4.146183,0.050984,0.070138,0.765961,-40.275967,-46.378803,9.059354,11.327907,...,1432.279419,895.710754,57.823505,2285.375732,1035.783569,66.249969,3243.091064,931.693176,57.935558,168.40
0 days 00:00:41.240000,0 days 00:00:41.260000,1.246349,46.816349,-10.047568,0.041397,0.075027,1.210429,-43.197887,-43.315212,6.661332,7.964739,...,1442.812500,496.417725,46.197857,2359.680176,1035.853638,59.971653,3314.937744,786.554382,57.275219,168.44
0 days 00:00:41.250000,0 days 00:00:41.270000,1.034742,44.519764,-6.430566,0.041330,0.076272,1.079818,-35.059437,-43.605213,3.711060,7.122712,...,1351.421021,568.486084,43.424572,2264.959717,910.367493,53.343594,3156.099609,828.785828,55.106598,168.48
0 days 00:00:41.260000,0 days 00:00:41.280000,0.662875,40.072189,6.419288,0.061406,0.074922,0.980079,-22.604826,-49.089691,-5.795834,1.717299,...,1293.338257,776.532166,39.866970,2253.175049,772.880371,46.456131,3265.535889,792.196594,50.720417,168.52
0 days 00:00:41.270000,0 days 00:00:41.290000,0.705874,36.022064,7.113981,0.066460,0.076228,0.849059,-25.321037,-52.743683,-8.227143,-5.869874,...,1172.029663,920.760437,51.670414,2193.498047,631.426270,48.063030,3271.723877,735.465088,53.065586,168.56
0 days 00:00:41.280000,0 days 00:00:41.300000,0.837876,38.638744,4.433558,0.070672,0.064587,0.847959,-37.392963,-39.446198,-12.236309,-12.688832,...,1174.511475,983.635681,52.589172,2287.624268,603.060364,49.006229,3503.480469,708.549561,56.510059,168.60
0 days 00:00:41.290000,0 days 00:00:41.310000,1.063633,41.986603,-8.854944,0.068933,0.061271,0.859786,-52.457340,-25.393885,-13.718742,-14.084976,...,1409.781372,158.503082,46.124020,2582.283936,677.956177,38.394970,3449.280029,990.414856,42.815693,168.64


In [18]:
df2 = pd.read_csv('/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/left-cam/left-cam-N249-planning-1-consolidated-face.csv')

In [44]:
df2.head(10)

,frame,child_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,0,34,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,32,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,34,0.00,0.88,1.0,-0.008382,0.412739,-0.910811,0.051284,0.487870,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3,1,32,0.00,0.88,1.0,-0.331415,0.146791,-0.931996,-0.348723,0.131543,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2,34,0.04,0.88,1.0,-0.069648,0.441450,-0.894579,0.005237,0.513091,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
5,2,32,0.04,0.88,1.0,-0.372041,0.173184,-0.911917,-0.541764,0.357923,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,3,34,0.08,0.98,1.0,-0.000444,0.523056,-0.852298,-0.361322,0.628799,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7,3,32,0.08,0.98,1.0,-0.467737,0.473355,-0.746430,-0.627960,0.459900,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,4,34,0.12,0.98,1.0,-0.043117,0.538832,-0.841309,-0.395897,0.661223,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
9,4,32,0.12,0.98,1.0,-0.516642,0.467105,-0.717561,-0.657745,0.394658,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df_merged = pd.merge(df, df2, on=['timestamp'])

In [42]:
df_merged.head(25).timestamp

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
Name: timestamp, dtype: float64

In [20]:
df.shape

(16851, 26)

In [62]:
len(x)

3725039

In [68]:
s = len(x) / sr
s = 169

In [71]:
df.shape[0] / 169 # 100ms window length, converted to 40ms (to match 25fps image extract)

24.964497041420117

In [119]:
d = pd.DataFrame(df)
d.describe().loc[['mean']].values.reshape(-1).tolist()



[0.06989282369613647,
 -8.527327537536621,
 17.965988159179688,
 0.041333284229040146,
 -0.005277508404105902,
 0.024462752044200897,
 15.884125709533691,
 -0.5284908413887024,
 -0.28954026103019714,
 -5.543805122375488,
 8.426116943359375,
 0.0075394934974610806,
 0.2868799567222595,
 1.9258490800857544,
 2.7280876636505127,
 7.23181676864624,
 571.5514526367188,
 1117.802490234375,
 -154.70651245117188,
 1562.2073974609375,
 -158.51040649414062,
 2568.493408203125,
 -160.51043701171875]

In [123]:
d.describe()

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz
count,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,...,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000,16851.000000
mean,0.069893,-8.527328,17.965988,0.041333,-0.005278,0.024463,15.884126,-0.528491,-0.289540,-5.543805,...,1.925849,2.728088,7.231817,571.551453,1117.802490,-154.706512,1562.207397,-158.510406,2568.493408,-160.510437
std,0.077506,11.266846,13.051626,0.047779,0.016128,0.086557,14.613583,13.052691,13.724513,13.259584,...,4.509568,12.115770,16.477356,183.062210,279.296844,79.081764,245.048508,71.725151,313.980713,68.139908
min,0.009305,-48.768559,-23.728790,-0.167433,-0.072691,0.000229,-31.020651,-46.180576,-40.971901,-56.799236,...,-16.368143,-196.770416,-178.040680,144.402084,172.835632,-201.000000,845.784729,-201.000000,1562.033325,-201.000000
25%,0.020508,-15.294378,8.286371,0.019654,-0.015748,0.001325,4.684283,-9.219364,-10.254114,-12.543498,...,0.000000,0.000000,0.000000,444.622192,927.044769,-201.000000,1399.174011,-201.000000,2354.411865,-201.000000
50%,0.039171,-5.249303,14.646319,0.045510,-0.003524,0.005243,12.471818,-2.762110,-3.260762,-3.378824,...,0.000000,0.000000,0.000000,512.674011,1105.980835,-201.000000,1498.024414,-201.000000,2469.239258,-201.000000
75%,0.095974,-0.119349,26.274628,0.071543,0.006513,0.021749,26.750667,6.752490,8.586273,2.887968,...,0.000000,0.000000,0.000000,681.194214,1295.963196,-133.102547,1689.735291,-137.077377,2729.369995,-138.328880
max,1.462344,23.967890,62.513088,0.183695,0.045366,3.694464,53.983704,47.570770,48.830204,36.587807,...,18.031408,197.832367,64.960396,1598.672363,3277.025391,22.257961,2669.953857,23.714699,3826.576416,28.335180


In [120]:
d[feature_set].mean().to_list()

[8.426129341125488,
 0.007539502345025539,
 0.286879301071167,
 1.9258522987365723,
 2.728087902069092,
 7.231820106506348,
 571.5498657226562,
 1117.8048095703125,
 -154.7063751220703,
 1562.204833984375,
 -158.51043701171875,
 2568.484619140625,
 -160.5103759765625]

In [128]:
d[feature_set].describe().loc['75p'].to_list()

[8.426116943359375,
 0.0075394934974610806,
 0.2868799567222595,
 1.9258490800857544,
 2.7280876636505127,
 7.23181676864624,
 571.5514526367188,
 1117.802490234375,
 -154.70651245117188,
 1562.2073974609375,
 -158.51040649414062,
 2568.493408203125,
 -160.51043701171875]

In [68]:
from utils import PROCESSED_AUD_FILE_PATH


audio_files=PROCESSED_AUD_FILE_PATH.glob('*.mp3')
audio_file_names=[i.stem for i in audio_files] 
audio_file_names.sort()

In [69]:
len(audio_file_names)

206

In [70]:
aud_mean_data = pd.DataFrame(columns=['pair_id'] + feature_set)

for i,audio_file in enumerate(audio_file_names):
    splitted_name = audio_file.split('-')  #['headset', 'audio', 'N249', 'planning', '1', 'child', '1', 'processed']

    x, sr = librosa.load(Path.joinpath(PROCESSED_AUD_FILE_PATH, audio_file + '.mp3'))
    aud_df = pd.DataFrame(extractor.process_signal(x,sr))
    
    aud_mean_data.loc[len(aud_mean_data)] = [splitted_name[2]] + aud_df[feature_set].mean().to_list()
    

In [71]:
aud_mean_data['condition'] = aud_mean_data['pair_id'].apply(lambda x: 1.0 if x[0] == 'P' else 0.0)
aud_mean_data.to_csv(DIR_PATH + 'binary-classification-audio/extracted-data/13fec_extract.csv')

In [115]:
aud_mean_data.describe()

,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,shimmerLocaldB_sma3nz,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz,condition
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,5.973370,0.004453,0.178509,1.090099,1.222297,3.546863,378.843636,558.960042,-169.286773,927.951415,-171.608016,1471.739806,-172.479460,0.524272
std,3.999503,0.003008,0.106924,0.939880,1.002359,2.732855,141.316377,224.337927,20.209670,334.080705,18.829301,527.644555,18.305890,0.500627
min,0.000000,0.000000,0.000000,-0.419415,-0.392948,-1.041932,50.681789,88.372711,-201.000000,135.052399,-201.000000,216.852524,-201.000000,0.000000
25%,2.842433,0.001941,0.086908,0.301275,0.488900,1.451738,297.753479,406.301117,-185.187550,704.929886,-186.633041,1134.013672,-187.237637,0.000000
50%,5.123663,0.003877,0.161218,0.921923,1.012978,2.779863,375.357468,534.695435,-172.616760,913.211151,-175.066757,1444.962280,-175.547150,1.000000
75%,9.075966,0.006611,0.259429,1.738087,1.710050,5.476543,461.412346,716.785370,-152.751617,1142.681091,-155.798405,1812.919830,-156.919842,1.000000
max,17.808149,0.012385,0.460987,4.486678,5.259802,13.454546,1054.315552,1387.801636,-114.957085,2110.470215,-119.664810,3168.084717,-121.214935,1.000000


In [109]:
aud_mean_data.describe()

,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,shimmerLocaldB_sma3nz,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz,condition
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,5.973370,0.004453,0.178509,1.090099,1.222297,3.546863,378.843636,558.960042,-169.286773,927.951415,-171.608016,1471.739806,-172.479460,0.524272
std,3.999503,0.003008,0.106924,0.939880,1.002359,2.732855,141.316377,224.337927,20.209670,334.080705,18.829301,527.644555,18.305890,0.500627
min,0.000000,0.000000,0.000000,-0.419415,-0.392948,-1.041932,50.681789,88.372711,-201.000000,135.052399,-201.000000,216.852524,-201.000000,0.000000
25%,2.842433,0.001941,0.086908,0.301275,0.488900,1.451738,297.753479,406.301117,-185.187550,704.929886,-186.633041,1134.013672,-187.237637,0.000000
50%,5.123663,0.003877,0.161218,0.921923,1.012978,2.779863,375.357468,534.695435,-172.616760,913.211151,-175.066757,1444.962280,-175.547150,1.000000
75%,9.075966,0.006611,0.259429,1.738087,1.710050,5.476543,461.412346,716.785370,-152.751617,1142.681091,-155.798405,1812.919830,-156.919842,1.000000
max,17.808149,0.012385,0.460987,4.486678,5.259802,13.454546,1054.315552,1387.801636,-114.957085,2110.470215,-119.664810,3168.084717,-121.214935,1.000000


In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.svm import SVC
import random

In [74]:
models = {
        "Logistic_Regression": {'function': LogisticRegression(), 'param_grid': [ {'solver': ['saga'], 'penalty': ['l1','l2'],
                                                                                   'tol': [1e-2, 1e-3, 1e-4, 1e-5]}]}, 
        
        "SVM": {'function': SVC(), 'param_grid': [{'kernel': ['poly'], 'degree': [2,3,4,5,6,7,8,9,10,15]},
                                                  {'kernel': ['linear', 'rbf', 'sigmoid']}]}, 
        
        "RF": {'function': RandomForestClassifier(), 'param_grid': [{'n_estimators': [2,5,10,20],'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [1,2,3,4,5,10,15,20,25],
                                                                                          'max_features': [0.1, 0.2, 'sqrt', 'log2']}]},
        "KNN":{'function': KNeighborsClassifier(), 'param_grid': [{'n_neighbors': [1,2,3,4,5,7,9]}]}
    }
    
    #Solver sag supports only 'l2' or 'none' penalties
    #Solver newton-cholesky supports only 'l2' or 'none' penalties

for name, items in models.items():
    train_acc_list = []
    test_acc_list = []
    k=10
    for i in range(k):
        pair_id_list = list(set(aud_mean_data["pair_id"]))  # get unique pair_ids
        random.shuffle(pair_id_list)

        x_train = aud_mean_data.loc[
            aud_mean_data["pair_id"].isin(pair_id_list[0 : int(0.70 * len(pair_id_list))])
        ]
        x_train = x_train[feature_set]
        y_train = aud_mean_data.loc[
            aud_mean_data["pair_id"].isin(pair_id_list[0 : int(0.70 * len(pair_id_list))])
        ]
        y_train = y_train["condition"]

        x_test = aud_mean_data.loc[
            aud_mean_data["pair_id"].isin(pair_id_list[int(0.7 * len(pair_id_list)) : -1])
        ]
        x_test = x_test[feature_set]
        y_test = aud_mean_data.loc[
            aud_mean_data["pair_id"].isin(pair_id_list[int(0.7 * len(pair_id_list)) : -1])
        ]
        y_test = y_test["condition"]

        best_model = GridSearchCV(items['function'], items['param_grid'])

        classifier = best_model.fit(x_train, y_train)

        pred_train, pred_test = classifier.predict(x_train), classifier.predict(
            x_test
        )

        train_acc, test_acc = accuracy_score(y_train, pred_train), accuracy_score(
            y_test, pred_test
        )

        train_acc_list.append(train_acc)

        test_acc_list.append(test_acc)
    
    print(f"{name}:-Best params - ", best_model.best_params_)

    print(
        f"{name}:- Mean train accuracy after {k}-fold cross validation: {np.mean(train_acc_list)}  (+/- {np.std(train_acc_list)}) "
    )
    print(
        f"{name}:- Mean test accuracy after {k}-fold cross validation: {np.mean(test_acc_list)}  (+/- {np.std(test_acc_list)})"
    )

/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/.venv/lib/python3.1

Logistic_Regression:-Best params -  {'l1_ratio': 0.0, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.01}
Logistic_Regression:- Mean train accuracy after 10-fold cross validation: 0.5703755015933939  (+/- 0.034237144991179476) 
Logistic_Regression:- Mean test accuracy after 10-fold cross validation: 0.48994657681080095  (+/- 0.10906034460218156)
SVM:-Best params -  {'degree': 15, 'kernel': 'poly'}
SVM:- Mean train accuracy after 10-fold cross validation: 0.6071712859968594  (+/- 0.0523949262651509) 
SVM:- Mean test accuracy after 10-fold cross validation: 0.40843359170389204  (+/- 0.06107165240878169)
RF:-Best params -  {'criterion': 'log_loss', 'max_depth': 5, 'max_features': 0.1, 'n_estimators': 10}
RF:- Mean train accuracy after 10-fold cross validation: 0.8258618255426278  (+/- 0.13763596142612725) 
RF:- Mean test accuracy after 10-fold cross validation: 0.4918349753694581  (+/- 0.09824267699650957)
KNN:-Best params -  {'n_neighbors': 5}
KNN:- Mean train accuracy after 10-fold cross va

In [108]:
for key, value in models.items():
    print(key, value['function'])

Logistic_Regression LogisticRegression()
SVM SVC()
RF RandomForestClassifier()
KNN KNeighborsClassifier()


In [4]:
from utils import FEATURE_SETS, extract_raw_data
features = [ 
                'mfcc_sma[1]',
                'mfcc_sma[2]',
                'mfcc_sma[3]',
                'mfcc_sma[4]',
                'mfcc_sma[5]',
                'mfcc_sma[6]',
                'mfcc_sma[7]',
                'mfcc_sma[8]',
                'mfcc_sma[9]',
                'mfcc_sma[10]',
                'mfcc_sma[11]',
                'mfcc_sma[12]',
               ]

In [3]:
featureset_names = ['GeM', 'eGe', 'emob'] #'GeM', 'eGe', 

In [5]:
for featureset in featureset_names:
    
    print(f"Feature set: {FEATURE_SETS[featureset]}\n")

    extractor = FEATURE_SETS[featureset]['extractor']
    
    features = FEATURE_SETS[featureset]["features"](extractor)
    
    #features = featureset['features'] #Custom features
    
    aud_mean_data = extract_raw_data(extractor, featureset, features, summary_stat = 'mean')


Feature set: {'extractor': {'$opensmile.core.smile.Smile': {'feature_set': 'GeMAPSv01b', 'feature_level': 'LowLevelDescriptors', 'options': {}, 'sampling_rate': None, 'channels': [0], 'mixdown': False, 'resample': False}}, 'features': <function <lambda> at 0x7fd555cf7490>}

Data does not exist in disk, reading audio files and extracting to a csv..

Extracted feature csv /media/chagan/2023-ELECTRA-field-MarcSchoolStudy/binary-classification-audio/extracted-data/summary-statistics-mean/GeM-extract.csv saved successfully!

Feature set: {'extractor': {'$opensmile.core.smile.Smile': {'feature_set': 'eGeMAPSv02', 'feature_level': 'LowLevelDescriptors', 'options': {}, 'sampling_rate': None, 'channels': [0], 'mixdown': False, 'resample': False}}, 'features': <function <lambda> at 0x7fd555cf7520>}

Data does not exist in disk, reading audio files and extracting to a csv..

Extracted feature csv /media/chagan/2023-ELECTRA-field-MarcSchoolStudy/binary-classification-audio/extracted-data/summary-s

In [91]:
results_df = pd.read_csv('/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/modality_fusion/inference/early_fusion/experiment-4/results_sorted_ck.csv')

#results_df = pd.read_csv('/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_video/inference/sklearn/experiment-9/results_df_test.csv')

In [95]:
results_df.rename(columns={"K": "K_fold", "CV_Val_score": "CV_Val_accuracy"}, inplace=True)

In [96]:
results_df.head(5)

,K_fold,Fold_number,Model,CV_Val_accuracy,CV_Val_RB,Test_accuracy_score,Test_precision_score,Test_recall_score,Test_F1_score,Test_Cohen_Kappa_score,Test_RB,Best_params,Aud_Features,Vid_Features
0,7,5,KNN,0.498413,0.510317,0.933333,0.944444,0.944444,0.944444,0.861111,0.6000,{'KNN__n_neighbors': 9},['pcm_loudness_sma'],"['AU06_c', 'AU12_c']"
1,8,5,RF,0.645866,0.525903,0.916667,1.000000,0.833333,0.909091,0.833333,0.5000,"{'RF__criterion': 'gini', 'RF__max_depth': 1, ...","['F0semitoneFrom27.5Hz_sma3nz', 'shimmerLocald...","['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06..."
2,6,1,RF,0.602614,0.517647,0.906250,0.857143,1.000000,0.923077,0.804878,0.5625,"{'RF__criterion': 'entropy', 'RF__max_depth': ...",['shimmerLocaldB_sma3nz'],"['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x..."
3,7,5,RF,0.538492,0.510317,0.900000,0.941176,0.888889,0.914286,0.794521,0.6000,"{'RF__criterion': 'log_loss', 'RF__max_depth':...","['pcm_loudness_sma', 'pcm_zcr_sma']","['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06..."
4,8,5,RF,0.610413,0.525903,0.875000,0.909091,0.833333,0.869565,0.750000,0.5000,"{'RF__criterion': 'gini', 'RF__max_depth': 1, ...","['mfcc_sma[1]', 'mfcc_sma[2]', 'mfcc_sma[3]', ...","['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06..."


In [100]:
def get_clean_foldwise_df(df, modality='Uni'):
       metrics = ['CV_Val_accuracy', 'CV_Val_RB','Test_accuracy_score', 'Test_precision_score', 'Test_recall_score',
              'Test_F1_score', 'Test_Cohen_Kappa_score','Test_RB']

       if modality == 'Uni':
              
              new_df = pd.DataFrame(columns=['K_fold', 'Model'] + [ 'Mean_'+ x for x in metrics ] + [ 'Std_'+ x for x in metrics ] + ['Features'])

              for K in df.K_fold.unique():
              
                     k_df = df.loc[ df['K_fold'] == K ]

                     for feat in k_df.Features.unique():
                            
                            fold_results_df_combined = k_df.loc[ (k_df['Features'] == feat) ]

                            for model in fold_results_df_combined.Model.unique():
                            
                                   fold_result_df_model = fold_results_df_combined.loc[ fold_results_df_combined['Model'] == model ]

                                   #print(fold_result_df_model.columns) #[metrics].mean()) #.loc['Test_accuracy_score']
                                   #print([K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [ fold_result_df_model[metrics].std().loc[x] for x in metrics ] + [feat])

                                   new_df.loc[len(new_df)] = [K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ]  + [ fold_result_df_model[metrics].std().loc[x] for x in metrics ] + [feat]

              new_df.sort_values(by=['Mean_Test_Cohen_Kappa_score'], ascending=False, inplace=True)

              return new_df
       else:
              new_df = pd.DataFrame(columns=['K_fold', 'Model'] + [ 'Mean_'+ x for x in metrics ] + [ 'Std_'+ x for x in metrics ]  + ['Aud_features', 'Vid_features'])
              
              for K in df.K_fold.unique():
              
                     k_df = df.loc[ df['K_fold'] == K ]

                     for aud_f, vid_f in zip(k_df.Aud_Features.unique(), k_df.Vid_Features.unique()):
                            
                            fold_results_df_combined = k_df.loc[ (k_df['Aud_Features']  == aud_f ) & (k_df['Vid_Features'] == vid_f) ]

                            for model in fold_results_df_combined.Model.unique():
                            
                                   fold_result_df_model = fold_results_df_combined.loc[ fold_results_df_combined['Model'] == model ]

                                   #print(fold_result_df_model.columns) #[metrics].mean()) #.loc['Test_accuracy_score']
                                   #print([K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [aud_f, vid_f])

                                   new_df.loc[len(new_df)] = [K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [ fold_result_df_model[metrics].std().loc[x] for x in metrics ]  + [aud_f, vid_f]
              
              new_df.sort_values(by=['Mean_Test_Cohen_Kappa_score'], ascending=False, inplace=True)           

              return new_df

In [101]:
cdf = get_clean_foldwise_df(results_df, modality='Multi')

In [90]:
cdf.head(10)

,K_fold,Model,Mean_CV_Val_accuracy,Mean_CV_Val_RB,Mean_Test_accuracy_score,Mean_Test_precision_score,Mean_Test_recall_score,Mean_Test_F1_score,Mean_Test_Cohen_Kappa_score,Mean_Test_RB,Std_CV_Val_accuracy,Std_CV_Val_RB,Std_Test_accuracy_score,Std_Test_precision_score,Std_Test_recall_score,Std_Test_F1_score,Std_Test_Cohen_Kappa_score,Std_Test_RB,Features
122,8,RF,0.651418,0.522665,0.661973,0.681424,0.686756,0.676351,0.319083,0.522665,0.027784,0.005541,0.137307,0.126861,0.172008,0.141154,0.271984,0.038790,"['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x..."
143,8,KNN,0.641762,0.522665,0.656193,0.695255,0.606027,0.641769,0.318507,0.522665,0.035260,0.005541,0.146744,0.166550,0.176662,0.162146,0.280614,0.038790,"['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', '..."
71,5,KNN,0.627928,0.524311,0.635440,0.676632,0.601364,0.633706,0.272672,0.524311,0.040193,0.007296,0.109848,0.118248,0.106982,0.103927,0.221833,0.029182,"['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', '..."
110,7,RF,0.677494,0.523129,0.626871,0.647430,0.640023,0.636526,0.250376,0.523129,0.015204,0.010692,0.058785,0.097430,0.122691,0.076717,0.122312,0.064149,"['AU06_c', 'AU12_c']"
39,4,KNN,0.644167,0.524231,0.627308,0.645149,0.708104,0.663087,0.241526,0.524231,0.034239,0.006116,0.098342,0.140507,0.160761,0.095578,0.200637,0.018349,"['AU06_c', 'AU12_c']"
129,8,SVM,0.603799,0.522665,0.614927,0.626237,0.707217,0.644448,0.231654,0.522665,0.030706,0.005541,0.107919,0.121791,0.227851,0.128691,0.208811,0.038790,"['AU06_r', 'AU12_r']"
23,3,KNN,0.595448,0.524370,0.611765,0.683612,0.518519,0.580795,0.230838,0.524370,0.083662,0.004367,0.017890,0.091854,0.089293,0.032270,0.045515,0.008733,"['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', '..."
105,7,SVM,0.583390,0.523129,0.614626,0.625428,0.677154,0.644508,0.230494,0.523129,0.045015,0.010692,0.167754,0.196707,0.146980,0.162466,0.329082,0.064149,"['AU06_r', 'AU12_r']"
111,7,KNN,0.642007,0.523129,0.615986,0.625379,0.715561,0.658791,0.228663,0.523129,0.019880,0.010692,0.092057,0.145832,0.104863,0.089086,0.188467,0.064149,"['AU06_c', 'AU12_c']"
38,4,RF,0.683205,0.524231,0.611731,0.641036,0.680632,0.635745,0.214783,0.524231,0.027794,0.006116,0.045661,0.095407,0.200836,0.091262,0.094224,0.018349,"['AU06_c', 'AU12_c']"


In [71]:
metrics = ['CV_Val_score', 'CV_Val_RB','Test_accuracy_score', 'Test_precision_score', 'Test_recall_score',
       'Test_F1_score', 'Test_Cohen_Kappa_score','Test_RB']

new_df = pd.DataFrame(columns=['K', 'Model'] + [ 'Mean_'+ x for x in metrics ] + [ 'Std_'+ x for x in metrics ] + ['Features'])

for K in results_df.K.unique():
    
    k_df = results_df.loc[ results_df['K'] == K ]

    for vid_f in k_df.Features.unique():
        
        fold_results_df_combined = k_df.loc[ (k_df['Features'] == vid_f) ]

        for model in fold_results_df_combined.Model.unique():
            
            fold_result_df_model = fold_results_df_combined.loc[ fold_results_df_combined['Model'] == model ]

            #print(fold_result_df_model.columns) #[metrics].mean()) #.loc['Test_accuracy_score']
            print([K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [ fold_result_df_model[metrics].std().loc[x] for x in metrics ] + [vid_f])

            new_df.loc[len(new_df)] = [K, model] + [ fold_result_df_model[metrics].std().loc[x] for x in metrics ] + [vid_f]

new_df.sort_values(by=['Mean_Test_Cohen_Kappa_score'], ascending=False, inplace=True)

[3, 'Logistic_Regression', 0.5245098039215687, 0.5243697478991596, 0.573249299719888, 0.5853641456582633, 0.638888888888889, 0.6043672447123579, 0.13942379459620835, 0.5243697478991596, "['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']"]
[3, 'SVM', 0.5901960784313726, 0.5243697478991596, 0.534593837535014, 0.6138418079096045, 0.5833333333333334, 0.5533539731682147, 0.06727957264570393, 0.5243697478991596, "['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']"]
[3, 'RF', 0.6590336134453781, 0.5243697478991596, 0.5015406162464986, 0.5090245775729647, 0.6574074074074074, 0.5642703337821415, -0.012217794044086136, 0.5243697478991596, "['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']"]
[3, 'KNN', 0.5760504201680672, 0.5243697478991596, 0.48613445378151265, 0.5325545540796964, 0.3796296296296296, 0.43438468037172623, -0.0166273192

In [72]:
new_df.sort_values(by=['Mean_Test_Cohen_Kappa_score'], ascending=False).head(5)

,K,Model,Mean_CV_Val_score,Mean_CV_Val_RB,Mean_Test_accuracy_score,Mean_Test_precision_score,Mean_Test_recall_score,Mean_Test_F1_score,Mean_Test_Cohen_Kappa_score,Mean_Test_RB,Features
94,6,RF,0.041644,0.004660,0.216535,0.281231,0.279807,0.278211,0.422070,0.023300,"['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', '..."
72,6,Logistic_Regression,0.092664,0.004660,0.209588,0.165205,0.332251,0.237828,0.412053,0.023300,"['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x..."
116,7,Logistic_Regression,0.067391,0.010692,0.215639,0.295100,0.277542,0.283573,0.411807,0.064149,"['pose_Tx', 'pose_Ty', 'pose_Tz', 'pose_Rx', '..."
96,7,Logistic_Regression,0.096897,0.010692,0.214886,0.214530,0.292892,0.249226,0.397808,0.064149,"['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x..."
88,6,Logistic_Regression,0.059849,0.004660,0.184318,0.141891,0.195631,0.159218,0.377193,0.023300,"['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06..."


In [66]:
metrics = ['CV_Val_score', 'CV_Val_RB','Test_accuracy_score', 'Test_precision_score', 'Test_recall_score',
       'Test_F1_score', 'Test_Cohen_Kappa_score','Test_RB']

new_df = pd.DataFrame(columns=['K', 'Model'] + [ 'Mean_'+ x for x in metrics ] + ['Aud_features', 'Vid_features'])
for K in results_df.K.unique():
    
    k_df = results_df.loc[ results_df['K'] == K ]

    for aud_f, vid_f in zip(k_df.Aud_Features.unique(), k_df.Vid_Features.unique()):
        
        fold_results_df_combined = k_df.loc[ (k_df['Aud_Features']  == aud_f ) & (k_df['Vid_Features'] == vid_f) ]

        for model in fold_results_df_combined.Model.unique():
            
            fold_result_df_model = fold_results_df_combined.loc[ fold_results_df_combined['Model'] == model ]

            #print(fold_result_df_model.columns) #[metrics].mean()) #.loc['Test_accuracy_score']
            print([K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [aud_f, vid_f])

            new_df.loc[len(new_df)] = [K, model] + [ fold_result_df_model[metrics].mean().loc[x] for x in metrics ] + [aud_f, vid_f]

            #print(f'{K} - {model} - ')


[7, 'KNN', 0.5981292517006802, 0.5231292517006801, 0.5704081632653061, 0.5681123731543899, 0.6119614512471655, 0.5873937532002048, 0.1327243904847841, 0.5231292517006803, "['pcm_loudness_sma']", "['AU06_c', 'AU12_c']"]
[7, 'Logistic_Regression', 0.5524943310657596, 0.5231292517006801, 0.5840136054421768, 0.6138365602651318, 0.5586734693877552, 0.5727504734857677, 0.16957325767664974, 0.5231292517006803, "['pcm_loudness_sma']", "['AU06_c', 'AU12_c']"]
[7, 'RF', 0.6645691609977324, 0.5231292517006801, 0.5965986394557824, 0.5954200867109737, 0.7814625850340137, 0.6640436048101552, 0.18467456467511814, 0.5231292517006803, "['pcm_loudness_sma']", "['AU06_c', 'AU12_c']"]
[7, 'SVM', 0.5880385487528345, 0.5231292517006801, 0.5027210884353741, 0.5383782883782884, 0.48880385487528344, 0.5011515928763507, -0.0006352044595861552, 0.5231292517006803, "['pcm_loudness_sma']", "['AU06_c', 'AU12_c']"]
[7, 'RF', 0.6043083900226758, 0.5231292517006801, 0.5639455782312924, 0.5765007821310342, 0.7138605442

In [67]:
new_df.sort_values(by=['Mean_Test_accuracy_score'], ascending=False).head(5)

,K,Model,Mean_CV_Val_score,Mean_CV_Val_RB,Mean_Test_accuracy_score,Mean_Test_precision_score,Mean_Test_recall_score,Mean_Test_F1_score,Mean_Test_Cohen_Kappa_score,Mean_Test_RB,Aud_features,Vid_features
80,4,RF,0.671987,0.524231,0.641923,0.634860,0.821429,0.707965,0.264864,0.524231,['shimmerLocaldB_sma3nz'],"['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06..."
84,4,KNN,0.625192,0.524231,0.632115,0.667789,0.675824,0.662875,0.259955,0.524231,"['pcm_loudness_sma', 'pcm_zcr_sma']","['AU06_c', 'AU12_c']"
20,8,RF,0.660150,0.522665,0.617960,0.631273,0.756324,0.669638,0.231056,0.522665,"['F0semitoneFrom27.5Hz_sma3nz', 'shimmerLocald...","['AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06..."
29,8,RF,0.664475,0.522665,0.610691,0.624440,0.739955,0.656825,0.220845,0.522665,['shimmerLocaldB_sma3nz'],"['AU06_c', 'AU12_c']"
92,4,RF,0.662115,0.524231,0.602500,0.593597,0.709478,0.644259,0.192319,0.524231,"['F0semitoneFrom27.5Hz_sma3nz', 'shimmerLocald...","['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x..."


In [48]:
metrics = ['CV_Val_score', 'CV_Val_RB','Test_accuracy_score', 'Test_precision_score', 'Test_recall_score',
       'Test_F1_score', 'Test_Cohen_Kappa_score','Test_RB']

print( [ 'Mean_'+ x for x in metrics ] )

['Mean_CV_Val_score', 'Mean_CV_Val_RB', 'Mean_Test_accuracy_score', 'Mean_Test_precision_score', 'Mean_Test_recall_score', 'Mean_Test_F1_score', 'Mean_Test_Cohen_Kappa_score', 'Mean_Test_RB']


In [ ]:
 '])
for K in results_df.K.unique():
    
    k_df = results_df.loc[ results_df['K'] == K ]

    for aud_f, vid_f in zip(k_df.Aud_Features.unique(), k_df.Vid_Features.unique()):
        
        fold_results_df_combined = k_df.loc[ (k_df['Aud_Features']  == aud_f ) & (k_df['Vid_Features'] == vid_f) ]

        for model in fold_results_df_combined.Model.unique():
            
            fold_result_df_model = fold_results_df_combined.loc[ fold_results_df_combined['Model'] == model ]

            print(fold_result_df_model[metrics].mean()) #.loc['Test_accuracy_score']

            break
        
        break
    
    break
            #print(f'{K} - {model} - ')
